# DVC Options - Metrics and Plots

In the following part we will look into metrics and plots from ZnTrack Nodes.
All `dvc run` options listed [here](https://dvc.org/doc/command-reference/run#options) can be used via `dvc.<option>`.
With the exception of params, which is handled automatically.
All these options take either `str` or `pathlib.Path` directed to the file the content should be stored in.
As shown before, `dvc.deps` can also take another `Node` as an argument.

In [1]:
from zntrack import Node, dvc, config
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
config.nb_name = "04_metrics_and_plots.ipynb"

In [3]:
!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/Documents/tmp/ZnTrack/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In the following we define a simple Node that produces a metric and a plot output using `json` and `pandas`.
We will queue multiple experiments with different outputs and then compare them afterwards.
With `@Node(silent=True)` we can reduce the amount of logs that will be displayed.

In [4]:
@Node(silent=True)
class MetricAndPlot:
    my_metric: Path = dvc.metrics(Path("my_metric.json"))
    my_plots: Path = dvc.plots("my_plots.csv")
    pre_factor = dvc.params()
    
    def __call__(self, pre_factor=1.0):
        self.pre_factor = pre_factor
    
    def run(self):
        self.my_metric.write_text(
            json.dumps({
                "metric_1": 17 * self.pre_factor,
                "metric_2": 42 * self.pre_factor
            })
        )
        
        x_data = np.linspace(0, 1.0 * self.pre_factor, 1000)
        y_data = np.exp(x_data)
        df = pd.DataFrame({"y": y_data, "x": x_data}).set_index("x")
        
        df.to_csv(self.my_plots)

2021-11-05 09:26:16,937 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


In [5]:
metric_and_plot = MetricAndPlot()
metric_and_plot()
!dvc repro
!git add .
!git commit -m "First Run"

Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True).run()"
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[master (root-commit) c833d9a] First Run
 18 files changed, 1707 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/simple.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore
 create mode 100644 .gitignore
 create mode 100644 .ipynb_checkpoints/03_metrics_and_plots-checkpoint.ipynb
 create mode 100644 03_metrics_and_plots.ipynb
 create mode 100644 config/zntrack.json
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
 create mode 100644 outs/.gitignore
 create mode 100644 src/MetricAndPlot.py
 create mode 100644 src/__pycache__/MetricAndPlot.cpython-39.pyc


In [6]:
metric_and_plot(pre_factor=2)
!dvc exp run --queue --name "factor_2"
metric_and_plot(pre_factor=3)
!dvc exp run --queue --name "factor_3"
metric_and_plot(pre_factor=4)
!dvc exp run --queue --name "factor_4"
metric_and_plot(pre_factor=5)
!dvc exp run --queue --name "factor_5"

Queued experiment '87be6eb' for future execution.
Queued experiment 'c7befa5' for future execution.
Queued experiment 'e47838f' for future execution.
Queued experiment '04febfd' for future execution.


In [7]:
!dvc exp run --run-all

Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True).run()"
Updating lock file 'dvc.lock'
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True).run()"
Updating lock file 'dvc.lock'
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True).run()"
Updating lock file 'dvc.lock'
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True).run()"
Updating lock file 'dvc.lock'

Reproduced experiment(s): factor_5, factor_4, factor_3, factor_2
To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>

To promote an experiment to a Git branch run:

	dvc exp branch <exp> <branch>



Now that all experiments are done, we can look at the metrics directly with `dvc exp show` or `dvc metrics show/diff`

In [8]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

,rev,typ,parent,metric_1,metric_2,MetricAndPlot.metrics.my_metric.Path,MetricAndPlot.plots.my_plots,MetricAndPlot.params.pre_factor,default
Experiment,,,,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,my_metric.json,my_plots.csv,5.0,NaN
master,c833d9a,baseline,NaN,17.0,42.0,my_metric.json,my_plots.csv,1.0,NaN
factor_2,ad1e284,branch_commit,NaN,34.0,84.0,my_metric.json,my_plots.csv,2.0,NaN
factor_3,fbfbcca,branch_commit,NaN,51.0,126.0,my_metric.json,my_plots.csv,3.0,NaN
factor_4,e0f77f7,branch_commit,NaN,68.0,168.0,my_metric.json,my_plots.csv,4.0,NaN
factor_5,305fb30,branch_base,NaN,85.0,210.0,my_metric.json,my_plots.csv,5.0,NaN


We can also use `dvc plots show/diff` to evaluate the plot data that we produced.

In [9]:
!dvc plots diff HEAD factor_2 factor_3 factor_4 factor_5

file:///C:/Users/fabia/Documents/tmp/ZnTrack/dvc_plots/index.html


In [1]:
from IPython.display import IFrame

In [2]:
IFrame(src='../../_static/plots/overview_04_plots.html', width=700, height=600)